# RETICOLO con passo 300

In [17]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import sys
from jax import grad
from jax import numpy as jnp

dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')

from giopy import Interpolazione, final_val

dir_path = os.path.abspath('')
FILE = dir_path + '/data/' + 'reticolo_300.csv'
fr = pd.read_csv(FILE)
fr = fr.set_index('colore')

{'n1 deg': 10, 'n1 prim': 20, 'n2 deg': 21, 'n2 prim': 10, 'n3 deg': 32, 'n3 prim': 30, '-n1 deg': 349, '-n1 prim': 40, '-n2 deg': 339, '-n2 prim': 5, '-n3 deg': 327, '-n3 prim': 32}


In [25]:
def formatter(data) -> dict:
    keys = ['n1','n2','n3','-n1','-n2','-n3']
    d = {}
    for i in range(0,len(data),2): # i = gradi, i+1 = primi
        d[keys[int(i/2)]] = np.float64(data[i] + data[i+1]/60)
    return d

giallo = formatter(fr.loc['GIALLO']) # come argomento passo una la riga
verde = formatter(fr.loc['VERDE'])
acqua = formatter(fr.loc['ACQUA'])